In [2]:
# validates randomly generated list-based ODs directly on the data
import pandas as pd
import random
from tqdm.notebook import tqdm
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta

In [3]:
def unique_counts(x: pd.Series):
    return len(x.unique())

def colsToString(cols: tuple[str], directions: tuple[bool]):
        return f",".join([col + ("↑" if direction else "↓") for col, direction in zip(cols, directions)])


class ListBasedDependency():
    def __init__(self, df: pd.DataFrame, lhs: list[str], lhsDirection: list[bool], rhs: list[str], rhsDirection : list[bool]) -> None:
        self.df = df
        self.lhs = lhs
        self.lhsDirection = lhsDirection
        self.rhs = rhs
        self.rhsDirection = rhsDirection
        
    # ensure propoer null first sorting by first sorting by _isna columns
    # returns (columns, direction)
    def create_sort_args(self, columns, directions):
        sort_cols = []
        sort_directions = []
        for col, direction in zip(columns, directions):
            sort_cols.append(col + "_isna")
            sort_directions.append(not direction)
            sort_cols.append(col)
            sort_directions.append(direction)
        return sort_cols, sort_directions
         
    
    def isValid(self):
        # no splits
        df_fd_check = self.df.groupby(self.lhs,dropna=False).agg({col: unique_counts for col in self.rhs})
        if not (df_fd_check == 1).all(axis=None):
             return False
        
        # no swaps
        lhs, lhsDirection = self.create_sort_args(self.lhs, self.lhsDirection)
        sorted_by_lhs = self.df.sort_values(lhs, ascending=lhsDirection)
        rhs, rhsDirection = self.create_sort_args(self.rhs, self.rhsDirection)
        sorted_by_rhs = sorted_by_lhs.sort_values(rhs, ascending=rhsDirection,kind="stable")
        return (sorted_by_lhs.index == sorted_by_rhs.index).all()
    
    def __str__(self):
        result = "["
        result += colsToString(self.lhs, self.lhsDirection)
        result += "] -> ["
        result += colsToString(self.rhs, self.rhsDirection)
        result += "]"
        return result    


In [4]:

def isPrefixOf(listA, listB):
    if(len(listB) < len(listA)):
        return False
    for i in range(len(listA)):
        if listA[i] != listB[i]:
            return False
    return True

def generateCnadidates(df, valid_amount = 100, max_side_length=6, max_tries = 10_000):
    valids = set()
    invalids = set()
    starttime = datetime.now()
    actual_cols = sorted(df.columns)
    max_side_length = min(max_side_length, len(actual_cols))
    df = pd.concat([df, df.isna().add_suffix("_isna")],axis=1)

    i = 0
    if i % 1000 == 0:
        print(f"{i=}")
    while len(valids) < valid_amount and i < max_tries:
        i += 1
        if datetime.now() - starttime > timedelta(minutes=30):
            print(f"Cancel after timeout with {len(valids)=} and {len(invalids)=}")
            return valids, invalids
        lhsSize = random.randint(1,max_side_length)
        rhsSize = random.randint(1,max_side_length)
        lhs = random.sample(actual_cols, lhsSize)
        rhs = random.sample(actual_cols, rhsSize)
        lhsDirection = [random.choice([True, False]) for _ in range(lhsSize)]
        rhsDirection = [random.choice([True, False]) for _ in range(rhsSize)]
        
        # skpi trivial ODs
        if isPrefixOf(rhsDirection, lhsDirection) or isPrefixOf(rhs,lhs):
            continue

        od = ListBasedDependency(df, lhs, lhsDirection, rhs, rhsDirection)

        if od.isValid():
            valids.add(str(od))

            if len(valids) % 10 == 0:
                print(f"{len(valids)=}")
            
        elif len(invalids) < 10_000:
            invalids.add(str(od))
    return valids, invalids

In [5]:
root_path = Path("/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs")

In [6]:
from itertools import product

null_values = ['','?']
null_permutations = [''.join(p) for p in product(*zip('null', 'NULL'))]
null_values.extend(null_permutations)

date_formats = ['%Y-%m-%dT%H:%M:%SZ', '%a, %d %b %Y %H:%M:%S %Z', '%m/%d/%Y', '%d.%m.%Y', '%Y-%m-%d', '%Y-%m-%d%z']

def load_data(file):
    df =  pd.read_csv(file,sep='\t',keep_default_na=False, na_values=null_values)
    for col in df.columns:
        if df[col].dtype == object:
            for date_format in date_formats:
                try:
                    df[col] = pd.to_datetime(df[col],format=date_format, errors='raise')
                    break
                except:
                    pass
    return df


In [7]:
from tqdm.notebook import tqdm

lens = []
result_path = Path('/Users/paulsieben/Programming/OrderDependencyTester/testdata')
for file in tqdm(list((root_path / "datasets").glob("**/*"))):
    if file.is_dir() or not file.name.endswith('.csv'): continue
    valid_path = result_path / "candidates" / str(file.parent.name)/ (str(file.name) + ".valids.txt")
    invalid_path = result_path / "candidates" / str(file.parent.name)/ (str(file.name) + ".invalids.txt")
    if valid_path.exists() and invalid_path.exists(): continue
    print(file)

    # df = load_data(file)
    # lens.append(len(df))

    valids, invalids = generateCnadidates(load_data(file))

    valid_path.parent.mkdir(parents=True, exist_ok=True)
    valid_path.write_text('\n'.join(valids))

    invalid_path.parent.mkdir(parents=True, exist_ok=True)
    invalid_path.write_text('\n'.join(invalids))
    


  0%|          | 0/144 [00:00<?, ?it/s]

/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/employee/salaries.csv
i=0
Cancel after timeout with len(valids)=0 and len(invalids)=9
/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/employee/dept_emp.csv
i=0
Cancel after timeout with len(valids)=0 and len(invalids)=6
/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/employee/dept_manager.csv
i=0
len(valids)=10
len(valids)=20
len(valids)=30
len(valids)=40
len(valids)=50
len(valids)=60
len(valids)=70
len(valids)=80
len(valids)=90
len(valids)=100
/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/employee/departments.csv
i=0
/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/employee/titles.csv
i=0
Cancel after timeout with len(valids)=0 and len(invalids)=22
/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and 

In [ ]:
# OD [8↓,3↓,12↓] -> [11↑]
df = pd.read_csv("/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/geneea/poslanec.csv", sep='\t')
od = ListBasedDependency(df, ["psc","id_kandidatka","psp_telefon"], [False, False, False], ["fax"], [True])
od.isValid()

In [ ]:
load_data("/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/geneea/organy.csv")

success od_organ %d.%m.%Y
success do_organ %d.%m.%Y


,id_organ,organ_id_organ,id_typ_organu,zkratka,nazev_organu_cz,nazev_organu_en,od_organ,do_organ,priorita,cl_organ_base
0,1,165.0,1,ČMSS,Poslanecký klub Českomoravské strany středu,Czech & Moravian Central Union,1994-02-15,1994-12-27,NaN,0.0
1,2,165.0,1,ČMUS,Poslanecký klub Českomoravské unie středu,Czech and Moravian Central Union,1992-06-06,1996-06-06,NaN,0.0
2,3,165.0,1,ČSSD,Poslanecký klub České strany sociálně demokrat...,Political group Czech Social Democratic Party,1992-06-06,1996-06-06,NaN,0.0
3,4,165.0,1,HSDMS,Poslanecký klub Hnutí za samosprávnou demokrac...,Movement for Self-Governing Democracy for Mora...,1992-06-06,1994-02-14,NaN,0.0
4,5,165.0,1,HSD-SMS,Poslanecký klub Hnutí za samosprávnou demokrac...,Movement for Self-Governing Democracy for Mora...,1992-06-06,1994-04-05,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
1177,1275,1140.0,13,PH,Filipínská republika,Republic of the Philippines,2015-04-10,NaT,NaN,0.0
1178,1276,1140.0,13,ID,Indonésie,Republic of Indonesia,2015-04-10,NaT,NaN,0.0
1179,1277,0.0,84,PSOBSE,Parlamentní shromáždění OBSE,NaN,1993-01-01,NaT,NaN,0.0
1180,1278,0.0,84,CPSOBSE,"Hlavní výbor pro demokracii, lidská práva a hu...",NaN,1993-01-01,NaT,NaN,1.0


In [71]:
# test [zkratka↓,od_organ↓,organ_id_organ↓,id_typ_organu↓] -> [zkratka↓,od_organ↓,cl_organ_base↓,do_organ↑]
# od_organ ist datum, parsingprobleme sind aber wahrscheinlich nicht ursächlich weil in lhs und rhs
# z 48 und 49, 61 und 62, 135 und 136, 160 161, 168 169, 194 195, 295 296 vertauscht 
df = pd.read_csv("/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/geneea/organy.csv", sep='\t')
od = ListBasedDependency(df, ["zkratka","od_organ","organ_id_organ","id_typ_organu"], [False, False, False, False], ["zkratka","od_organ","cl_organ_base","do_organ"], [False, False, False, True])
od.isValid()

False

In [22]:
df = load_data("/Users/paulsieben/HPI/WiSe 2023-2024 Advanced Data Profiling/Example Data and ODs/datasets/geneea/poslanec.csv")

In [34]:
ListBasedDependency(df, ["id_poslanec"], [False], ["fax"], [True]).isValid()

False